## Tensorflow 2.0 Quickstart for Beginners

This short introduction uses Keras to:

- Build a neural network that classifies images.
- Train this neural network.
- And, finally, evaluate the accuracy of the model.

Import TensorFlow into your program:

In [1]:
import tensorflow as tf

print(tf.__version__)

2.4.1


Load and prepare the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset. Convert the samples from integers to floating-point numbers:


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

Build the [`tf.keras.Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.


In [5]:
predictions = model(x_train[:1]).numpy()
print(predictions)

[[-0.48228213 -0.10659518  0.49774444  0.3015085   0.38520667  0.36116505
  -0.1141517  -0.06780333 -0.3868947   0.2432563 ]]


The [`tf.nn.softmax`](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) function converts these logits to "probabilities" for each class:

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.05512647, 0.08026366, 0.14688596, 0.12071357, 0.13125195,
        0.12813407, 0.07965942, 0.08343841, 0.0606438 , 0.11388263]],
      dtype=float32)

**Note**: It is possible to bake this tf.nn.softmax in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

The [`losses.SparseCategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy) loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.0546782

In [9]:
model.compile(
    optimizer="adam",
    loss=loss_fn,
    metrics=["accuracy"]
)

The [`Model.fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) method adjusts the model parameters to minimize the loss:

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 775us/step - loss: 0.4863 - accuracy: 0.8614
Epoch 2/5
1875/1875 [==============================] - 1s 762us/step - loss: 0.1485 - accuracy: 0.9565
Epoch 3/5
1875/1875 [==============================] - 1s 713us/step - loss: 0.1122 - accuracy: 0.9658
Epoch 4/5
1875/1875 [==============================] - 1s 742us/step - loss: 0.0854 - accuracy: 0.9740
Epoch 5/5
1875/1875 [==============================] - 1s 747us/step - loss: 0.0712 - accuracy: 0.9777


The [`Model.evaluate`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate) method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [11]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0744 - accuracy: 0.9784


[0.0744459256529808, 0.9783999919891357]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [18]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[7.7327851e-09, 4.4234385e-09, 5.7774751e-06, 3.8170067e-04,
        2.3066035e-11, 1.6040967e-07, 3.7301034e-14, 9.9959058e-01,
        1.6376148e-06, 2.0215277e-05],
       [7.0038936e-10, 1.0013861e-05, 9.9997628e-01, 1.0792411e-05,
        1.3037892e-12, 1.6026181e-06, 4.4553459e-09, 7.0807412e-13,
        1.3041762e-06, 2.4441759e-12],
       [4.6188006e-06, 9.9217927e-01, 3.8730510e-04, 2.4378934e-04,
        7.3648021e-05, 1.1718879e-04, 1.0003882e-04, 5.4163807e-03,
        1.4750303e-03, 2.7722333e-06],
       [9.9986434e-01, 7.9195736e-08, 2.4711726e-05, 4.3353818e-08,
        3.5894277e-06, 4.3178666e-06, 2.4104542e-05, 6.4741973e-05,
        4.6096062e-08, 1.4052387e-05],
       [2.4304535e-07, 1.6609134e-10, 3.8889723e-07, 4.1721147e-09,
        9.8705626e-01, 1.5645790e-07, 1.1492004e-06, 1.5682015e-05,
        1.7482392e-07, 1.2925992e-02]], dtype=float32)>

## Extra

Additionally, if you want to find to corresponding class prediction labels, we can compute the argmax:

In [16]:
probability_model(x_test[:5]).numpy().argmax(axis=1)

array([7, 2, 1, 0, 4])

When compared to the true labels, we see the model performed rather well.

In [17]:
y_test[:5]

array([7, 2, 1, 0, 4], dtype=uint8)